#### stack()

Stacking levels from column to row. 

After compressing a level, it either produces a `series` in case of a simple column Index or a `dataframe` if MultiIndex in the columns. 

Incase if level of column that is to be stacked not mentioned, then automatically last level is taken. 

If the columns have a MultiIndex, we can choose which level to stack. The stacked level becomes the new lowest level in a MultiIndex on the columns.

In [1]:
import pandas as pd
import numpy as np
df = pd.DataFrame({'A': range(1, 6),
                   'B': range(10, 0, -2),
                   'C_C': range(10, 5, -1)})
df

,A,B,C_C
0,1,10,10
1,2,8,9
2,3,6,8
3,4,4,7
4,5,2,6


In [2]:
df.stack() 
# simply stacks the label from column to row. 
# as this is a simple column index, it produces a pandas Series.  

0  A       1
   B      10
   C_C    10
1  A       2
   B       8
   C_C     9
2  A       3
   B       6
   C_C     8
3  A       4
   B       4
   C_C     7
4  A       5
   B       2
   C_C     6
dtype: int64

In [3]:
multi_col_1 = pd.MultiIndex.from_tuples(
    [('Wind', 'mph'), ('Wind', 'm/s')]
)
df_multi_level_1 = pd.DataFrame(
    [[13, 5.5], [19, 8.5]],
    index=['London', 'Oxford'],
    columns=multi_col_1
)

df_multi_level_1

Wind     
        mph  m/s
London   13  5.5
Oxford   19  8.5

In [4]:
df_multi_level_1.stack(level= -1) # eq to stack(-1)
# as the last column level is 'mph' and	'm/s', stack() takes these column levels and stack them along row axis
# this is a df with MultiIndex columns. 
# Produces a dataframe as it contains MultiIndex columns

Wind
London m/s   5.5
       mph  13.0
Oxford m/s   8.5
       mph  19.0

In [5]:
# Similarly we can pass list of levels or list of index of those levels too
columns = pd.MultiIndex.from_tuples(
    [
        ("A", "cat", "long"),
        ("B", "cat", "long"),
        ("A", "dog", "short"),
        ("B", "dog", "short"),
    ],
    names=["exp", "animal", "hair_length"],
)


df = pd.DataFrame(np.random.randn(4, 4), columns=columns)

df2 = df.stack(level=[1,2]) # Equivalent to level=["animal", "hair_length"]
df2

exp                          A         B
  animal hair_length                    
0 cat    long        -2.285684 -0.315318
  dog    short       -0.231337  1.604305
1 cat    long         0.237562 -0.183423
  dog    short        0.439756  0.356043
2 cat    long        -0.191487  1.777393
  dog    short        0.206887  1.281781
3 cat    long         0.325672  0.292657
  dog    short       -0.627474 -1.703425

In [6]:
# Missing values are filled with NaN automatically. 
# Can happen during both stacking and unstacking
# unstacking takes an argument called 'fill_value' to replace the NaN with
df2.unstack('hair_length')

exp                 A                   B          
hair_length      long     short      long     short
  animal                                           
0 cat       -2.285684       NaN -0.315318       NaN
  dog             NaN -0.231337       NaN  1.604305
1 cat        0.237562       NaN -0.183423       NaN
  dog             NaN  0.439756       NaN  0.356043
2 cat       -0.191487       NaN  1.777393       NaN
  dog             NaN  0.206887       NaN  1.281781
3 cat        0.325672       NaN  0.292657       NaN
  dog             NaN -0.627474       NaN -1.703425

#### unstack()

Opposite of stacking.

Level can be choosen to unstack. default is last level or -1 which will apply the operation on the inner-most index.

Takes an argument called 'fill_value' to replace the NaN with.

If passing a list of index levels, order of levels are important for getting a unstaked df

In [14]:
columns = pd.MultiIndex.from_tuples(
    [
        ("A", "cat"),
        ("B", "dog"),
        ("B", "cat"),
        ("A", "dog"),
    ],
    names=["exp", "animal"],
)

display(df)

index = pd.MultiIndex.from_product(
    [("bar", "baz", "foo", "qux"), ("one", "two")], names=["first", "second"]
)

df.unstack(0)

exp,A,B,A,B
animal,cat,cat,dog,dog
hair_length,long,long,short,short
0,-2.285684,-0.315318,-0.231337,1.604305
1,0.237562,-0.183423,0.439756,0.356043
2,-0.191487,1.777393,0.206887,1.281781
3,0.325672,0.292657,-0.627474,-1.703425


exp  animal  hair_length   
A    cat     long         0   -2.285684
                          1    0.237562
                          2   -0.191487
                          3    0.325672
B    cat     long         0   -0.315318
                          1   -0.183423
                          2    1.777393
                          3    0.292657
A    dog     short        0   -0.231337
                          1    0.439756
                          2    0.206887
                          3   -0.627474
B    dog     short        0    1.604305
                          1    0.356043
                          2    1.281781
                          3   -1.703425
dtype: float64

In [32]:
index = pd.MultiIndex.from_tuples([
  ('Oxford', 'Weather', '01-01-2022'), 
  ('Oxford', 'Temperature', '01-01-2022'), 
  ('Oxford', 'Weather', '02-01-2022'),
  ('Oxford', 'Temperature', '02-01-2022'),
  ('London', 'Weather', '01-01-2022'), 
  ('London', 'Temperature', '01-01-2022'),
  ('London', 'Weather', '02-01-2022'),
  ('London', 'Temperature', '02-01-2022'),
])
s = pd.Series(
  ['Sunny', 10, 'Shower', 7, 'Shower', 5, 'Sunny', 8], 
  index=index
)

s.unstack() # Unstacking by default with the innermost index 

01-01-2022 02-01-2022
London Temperature          5          8
       Weather         Shower      Sunny
Oxford Temperature         10          7
       Weather          Sunny     Shower

In [33]:
# we can pass chain of unstack() method to apply unstacking simentanously 
s.unstack().unstack()

01-01-2022          02-01-2022        
       Temperature Weather Temperature Weather
London           5  Shower           8   Sunny
Oxford          10   Sunny           7  Shower

In [34]:
# We can also pass list of index level to unstack. Remember the order of passing index levels is important 
s.unstack([1,2])

,Weather,Temperature,Weather,Temperature
,01-01-2022,01-01-2022,02-01-2022,02-01-2022
London,Shower,5,Sunny,8
Oxford,Sunny,10,Shower,7


In [35]:
# Imdex levels passed in different order 
s.unstack([2,1])

01-01-2022             02-01-2022            
          Weather Temperature    Weather Temperature
London     Shower           5      Sunny           8
Oxford      Sunny          10     Shower           7